In [1]:
'''
When working with ecoinvent 3.8, I need to use brightway2 + a downgrade of bw2io in order to properly create the biosphere3 database and to properly load ecoinvent 3.8 cutoff

The ecoinvent 3.8 cutoff dataset has been loaded in the previous notebook, so now we need to use a different environment:
The environment setup that can do this is bw2premise-ei38compat-ok

In this notebook, we'll be using brightway2.5
'''

"\nWhen working with ecoinvent 3.8, I need to use brightway2 + a downgrade of bw2io in order to properly create the biosphere3 database and to properly load ecoinvent 3.8 cutoff\n\nThe ecoinvent 3.8 cutoff dataset has been loaded in the previous notebook, so now we need to use a different environment:\nThe environment setup that can do this is bw2premise-ei38compat-ok\n\nIn this notebook, we'll be using brightway2.5\n"

# Libraries

In [1]:
# Import BW25 packages.
import bw2data as bd
import copy
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa

import matplotlib.pyplot as plt

# 1. Support Functions

In [2]:
from database_setup import scenario_db_name

# 2. Imports and Declarations

In [3]:
bd.projects

Brightway2 projects manager with 12 objects:
	LNV-EI38
	LNV-EI38-20241204
	LNV-EI38-test20241120
	LNV-EI38-test20241120-BaselineEI38
	V1_pLCA_LiIon
	V1_pLCA_liIon
	V2_pLCA_LiIon
	V2_pLCA_liIon
	V3_pLCA_liion
	default
	test-EI38-SSP1-Base
	v3_liion
Use `projects.report()` to get a report on all projects.

In [4]:
#Creating/accessing the project
bd.projects.set_current("LNV-EI38-20241204")

# 3. Incorporating Background Scenarios with Premise

In [5]:
# From here:
# https://premise.readthedocs.io/en/latest/introduction.html#default-iam-scenarios
from premise import *

In [ ]:
'''
We'll incorporate the following scenarios, each per database:
- SSP1-None (REMIND SSP1-Base)
- SSP1-RCP1.9 (REMIND SSP1-PkBudg500)
- SSP1-RCP2.6 (REMIND SSP1-PkBudg1150)
- SSP2-None (REMIND SSP2-Base)
- SSP2-RCP1.9 (REMIND SSP2-PkBudg500)
- SSP2-RCP2.6 (REMIND SSP2-PkBudg1150)
- SSP5-Base

From:
2025 to 2040

And Create:
- Baseline (keep without transforming)
- BL + VSI
'''

## 2.1. Defining the scenarios

In [6]:
# I want to work in between 2025 and 2040, therefore:
years = range(2020, 2041, 5)

In [7]:
# The scenarios defined previously are:
scenario_definitions = [
    {"model": "remind", "pathway": "SSP1-Base"}, # Confirmed working
    # {"model": "remind", "pathway": "SSP1-PkBudg500"}, 
    # {"model": "remind", "pathway": "SSP1-PkBudg1150"},
    {"model": "remind", "pathway": "SSP2-Base"}, # Confirmed working
    # {"model": "remind", "pathway": "SSP2-PkBudg500"}, # Confirmed working
    # {"model": "remind", "pathway": "SSP2-PkBudg1150"}, # Confirmed working
    {"model": "remind", "pathway": "SSP5-Base"}, # Confirmed working
]

In [8]:
# Generating my list:
scenarios = [
    {**scenario, "year": year}
    for scenario in scenario_definitions
    for year in years
]

# List of custom database names with a suffix
suffix_to_include = 'baseline'
db_names = [scenario_db_name(scenario["model"], scenario["pathway"], scenario["year"], suffix=suffix_to_include) for scenario in scenarios]

In [9]:
scenarios

[{'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2020},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2025},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2030},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2035},
 {'model': 'remind', 'pathway': 'SSP1-Base', 'year': 2040},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2020},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2025},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2030},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2035},
 {'model': 'remind', 'pathway': 'SSP2-Base', 'year': 2040},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2020},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2025},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2030},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2035},
 {'model': 'remind', 'pathway': 'SSP5-Base', 'year': 2040}]

In [10]:
# Creating the new databases:
ndb = NewDatabase(
      scenarios=scenarios,
      source_db="ecoinvent 3.8 cutoff",
      source_version="3.8", 
      key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
      use_multiprocessing=True,
      keep_uncertainty_data=False
  )

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [ ]:
'''
Up to this point, we'll have the new databases (defined by the scenarios) added to the project.

We still need to transform the database based on the updates we want to run.

Some update examples here:
https://github.com/polca/premise/blob/master/examples/examples.ipynb

List of ndb.update() methods:
https://github.com/polca/premise/blob/3bb7a9c9bc0b72165450f27c3dce1a2cbedd523e/premise/new_database.py#L866

'''

In [11]:
ndb.update() # <- updates eveyrthing!

## More info here:
# https://premise.readthedocs.io/en/latest/transform.html

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: external:  90%|███████████▋ | 9/10 [22:07<02:27, 147.48s/it]


External scenarios (datapackages) are not provided. Skipped.
Done!



In [12]:
db_names

['EI38_cutoff_remind_SSP1-Base_2020_baseline',
 'EI38_cutoff_remind_SSP1-Base_2025_baseline',
 'EI38_cutoff_remind_SSP1-Base_2030_baseline',
 'EI38_cutoff_remind_SSP1-Base_2035_baseline',
 'EI38_cutoff_remind_SSP1-Base_2040_baseline',
 'EI38_cutoff_remind_SSP2-Base_2020_baseline',
 'EI38_cutoff_remind_SSP2-Base_2025_baseline',
 'EI38_cutoff_remind_SSP2-Base_2030_baseline',
 'EI38_cutoff_remind_SSP2-Base_2035_baseline',
 'EI38_cutoff_remind_SSP2-Base_2040_baseline',
 'EI38_cutoff_remind_SSP5-Base_2020_baseline',
 'EI38_cutoff_remind_SSP5-Base_2025_baseline',
 'EI38_cutoff_remind_SSP5-Base_2030_baseline',
 'EI38_cutoff_remind_SSP5-Base_2035_baseline',
 'EI38_cutoff_remind_SSP5-Base_2040_baseline']

In [13]:
# Writing the databases to Brightway 
## Considering this to be the baseline
ndb.write_db_to_brightway(name=db_names)

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:45


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 17:33:26
  Finished: 12/04/2024 17:34:11
  Total time elapsed: 00:00:45
  CPU %: 17.90
  Memory %: 40.36
Created database: EI38_cutoff_remind_SSP1-Base_2020_baseline


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 17:36:50
  Finished: 12/04/2024 17:37:33
  Total time elapsed: 00:00:43
  CPU %: 28.60
  Memory %: 40.54
Created database: EI38_cutoff_remind_SSP1-Base_2025_baseline


...

Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 18:19:20
  Finished: 12/04/2024 18:20:04
  Total time elapsed: 00:00:43
  CPU %: 25.00
  Memory %: 41.36
Created database: EI38_cutoff_remind_SSP5-Base_2030_baseline


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:48


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 18:23:42
  Finished: 12/04/2024 18:24:30
  Total time elapsed: 00:00:48
  CPU %: 17.70
  Memory %: 41.40
Created database: EI38_cutoff_remind_SSP5-Base_2035_baseline


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 18:28:11
  Finished: 12/04/2024 18:28:55
  Total time elapsed: 00:00:44
  CPU %: 11.40
  Memory %: 41.44
Created database: EI38_cutoff_remind_SSP5-Base_2040_baseline
Generate scenario report.
Report saved under g:\My Drive\10. PHD\001. Articles\2024_LCIA_DRAFT\2 - Analysis\V7\export\scenario_report.
Generate change report.
Report saved under g:\My Drive\10. PHD\001. Articles\2024_LCIA_DRAFT\2 - Analysis\V7.


# 3. Also creating the VSI anciliary database

In [14]:
# List of custom database names with a suffix
suffix_to_include = 'VSI'
db_names = [scenario_db_name(scenario["model"], scenario["pathway"], scenario["year"], suffix=suffix_to_include) for scenario in scenarios]

In [15]:
# Creating the new databases:
ndb = NewDatabase(
      scenarios=scenarios,
      source_db="ecoinvent 3.8 cutoff",
      source_version="3.8", 
      key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
      use_multiprocessing=True,
      keep_uncertainty_data=False
  )

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [16]:
ndb.update() # <- updates eveyrthing!

## More info here:
# https://premise.readthedocs.io/en/latest/transform.html

`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: external:  90%|███████████▋ | 9/10 [30:02<03:20, 200.28s/it]


External scenarios (datapackages) are not provided. Skipped.
Done!



In [17]:
# Writing the databases to Brightway 
## Considering this to be the baseline
ndb.write_db_to_brightway(name=db_names)

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:51


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 19:14:24
  Finished: 12/04/2024 19:15:15
  Total time elapsed: 00:00:51
  CPU %: 27.10
  Memory %: 55.28
Created database: EI38_cutoff_remind_SSP1-Base_2020_VSI


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:45


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 19:23:05
  Finished: 12/04/2024 19:23:50
  Total time elapsed: 00:00:45
  CPU %: 13.10
  Memory %: 53.90
Created database: EI38_cutoff_remind_SSP1-Base_2025_VSI


...

Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 20:45:27
  Finished: 12/04/2024 20:46:12
  Total time elapsed: 00:00:44
  CPU %: 11.30
  Memory %: 30.16
Created database: EI38_cutoff_remind_SSP5-Base_2030_VSI


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:45


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 20:56:47
  Finished: 12/04/2024 20:57:32
  Total time elapsed: 00:00:45
  CPU %: 13.00
  Memory %: 31.20
Created database: EI38_cutoff_remind_SSP5-Base_2035_VSI


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:50


Title: Writing activities to SQLite3 database:
  Started: 12/04/2024 21:09:44
  Finished: 12/04/2024 21:10:34
  Total time elapsed: 00:00:50
  CPU %: 13.10
  Memory %: 31.54
Created database: EI38_cutoff_remind_SSP5-Base_2040_VSI
Generate scenario report.
Report saved under g:\My Drive\10. PHD\001. Articles\2024_LCIA_DRAFT\2 - Analysis\V7\export\scenario_report.
Generate change report.
Report saved under g:\My Drive\10. PHD\001. Articles\2024_LCIA_DRAFT\2 - Analysis\V7.


In [18]:
list(bd.databases)

['biosphere3',
 'ecoinvent 3.8 cutoff',
 'EI38_cutoff_remind_SSP1-Base_2020_baseline',
 'EI38_cutoff_remind_SSP1-Base_2025_baseline',
 'EI38_cutoff_remind_SSP1-Base_2030_baseline',
 'EI38_cutoff_remind_SSP1-Base_2035_baseline',
 'EI38_cutoff_remind_SSP1-Base_2040_baseline',
 'EI38_cutoff_remind_SSP2-Base_2020_baseline',
 'EI38_cutoff_remind_SSP2-Base_2025_baseline',
 'EI38_cutoff_remind_SSP2-Base_2030_baseline',
 'EI38_cutoff_remind_SSP2-Base_2035_baseline',
 'EI38_cutoff_remind_SSP2-Base_2040_baseline',
 'EI38_cutoff_remind_SSP5-Base_2020_baseline',
 'EI38_cutoff_remind_SSP5-Base_2025_baseline',
 'EI38_cutoff_remind_SSP5-Base_2030_baseline',
 'EI38_cutoff_remind_SSP5-Base_2035_baseline',
 'EI38_cutoff_remind_SSP5-Base_2040_baseline',
 'EI38_cutoff_remind_SSP1-Base_2020_VSI',
 'EI38_cutoff_remind_SSP1-Base_2025_VSI',
 'EI38_cutoff_remind_SSP1-Base_2030_VSI',
 'EI38_cutoff_remind_SSP1-Base_2035_VSI',
 'EI38_cutoff_remind_SSP1-Base_2040_VSI',
 'EI38_cutoff_remind_SSP2-Base_2020_VSI',
 'E